In [1]:
# ====== SETUP + INSTALL ======
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Dự án - báo cáo/Do_an_HK3_DT2210L_Text_To_SQL/vi_t5_text2sql

/content/drive/MyDrive/Dự án - báo cáo/Do_an_HK3_DT2210L_Text_To_SQL/vi_t5_text2sql


In [3]:
# ======== Import Libraries ========
!pip install tabulate transformers gradio
!pip install Unidecode
import gradio as gr
import re
import json
import pandas as pd
import sqlite3
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tabulate import tabulate
from unidecode import unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.0 MB/s eta 0:00:00


In [4]:
# ======= CONFIG ========
model_dir = "model/Final_model"
db_path = "data/processing/SQLite_real_estate.db"
schema = "address[str], area[float], frontage[float], access_road[float], house_direction[str], balcony_direction[str], \
          floors[int], bedrooms[int], bathrooms[int], legal_status[str], furniture_state[str], price[float], city[str], district[str], \
          ward[str], cluster_label[str]"

# Load nested JSON
with open("data/processing/locations.json", "r", encoding="utf-8") as f:
    nested = json.load(f)

# Flatten: từ {city: {district: [ward, ...]}} → list of dict
flat = []
for city, districts in nested.items():
    for district, wards in districts.items():
        for ward in wards:
            flat.append({
                "city": city,
                "district": district,
                "ward": ward
            })

# Save file JSON
with open("data/processing/locations_flat.json", "w", encoding="utf-8") as f:
    json.dump(flat, f, ensure_ascii=False, indent=2)

# Optionally: convert to CSV để dễ nhìn
pd.DataFrame(flat).to_csv("data/processing/locations_flat.csv", index=False)
filepath_location = "data/processing/locations_flat.json"

# ======= LOAD MODEL ========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.eval().to(device)

# ======= STEP 1: Xử lý câu hỏi đầu vào ========
full_schema = [tuple(col.strip().split("[")) for col in schema.split(",")]
full_schema = [(col, dtype.strip("]")) for col, dtype in full_schema]

SCHEMA_KEYWORDS = {
    'address': ['địa chỉ'], 'area': ['diện tích', 'm2', 'mét vuông'],
    'price': ['giá', 'tỷ', 'triệu', 'bao nhiêu tiền'],
    'frontage': ['mặt tiền'], 'access_road': ['đường vào', 'đường', 'hẻm'],
    'house_direction': ['hướng nhà'], 'balcony_direction': ['hướng ban công'],
    'floors': ['tầng', 'lầu'], 'bedrooms': ['phòng ngủ'], 'bathrooms': ['phòng tắm', 'wc'],
    'legal_status': ['pháp lý', 'sổ hồng', 'sổ đỏ'], 'furniture_state': ['nội thất'],
    'city': ['thành phố'], 'district': ['quận', 'huyện'], 'ward': ['phường', 'xã'],
    'cluster_label': ['phân khúc']
}

def extract_relevant_columns_from_question(question: str, schema: list) -> list:
    question = question.lower()
    relevant = []
    for col, dtype in schema:
        if col in SCHEMA_KEYWORDS:
            for kw in SCHEMA_KEYWORDS[col]:
                if kw in question:
                    relevant.append((col, dtype))
                    break
    if 'giá' in question and ('price', 'float') not in relevant:
        relevant.append(('price', 'float'))
    return relevant

def generate_input_text(question: str, schema_columns: list) -> str:
    schema_str = ", ".join([f"{col}[{dtype}]" for col, dtype in schema_columns])
    return f"Câu hỏi: {question} | Schema: {schema_str}"

def normalize_question(text: str) -> str:
    """
    Normalize a natural language question:
    - Convert to lowercase and strip whitespace
    - Convert expressions like "2.5 tỷ", "3 triệu" to numeric VND values
    - Normalize city name variants (e.g., SG, HCM → hồ chí minh)
    """
    try:
        text = text.lower().strip()

        # Biến thể của thành phố Hồ Chí Minh
        hcm_aliases = [
            "thành phố hồ chí minh", "tp hồ chí minh", "tp. hồ chí minh", "tphcm",
            "tp hcm", "tp.hcm", "hcm", "hcm city",
            "sài gòn", "tp sg", "thành phố sg", "sg", "saigon"
        ]

        # Thay các alias trong câu hỏi thành 'hồ chí minh'
        for alias in hcm_aliases:
            if alias in text:
                text = text.replace(alias, "hồ chí minh")

        return text
    except Exception as e:
        print(f"Error in normalize_question: {e}")
        return text

def load_locations(filepath: str):
    try:
        df = pd.read_csv(filepath) if filepath.endswith(".csv") else pd.read_json(filepath)
        return df[['city', 'district', 'ward']].dropna().drop_duplicates().to_dict(orient="records")
    except Exception as e:
        print(f"Error loading locations: {e}")
        return []

locations = load_locations(filepath_location)
def extract_location_from_question_v2(question: str, locations: list) -> dict:
    try:
        question = unidecode(question.lower())
        matched = {'city': None, 'district': None, 'ward': None}

        # Ưu tiên match ward + đúng district nếu district có xuất hiện trong câu hỏi
        for loc in locations:
            ward = unidecode(loc['ward'].lower())
            district = unidecode(loc['district'].lower())
            if re.search(rf'\b{re.escape(ward)}\b', question):
                if re.search(rf'\b{re.escape(district)}\b', question):
                    return {'city': loc['city'], 'district': loc['district'], 'ward': loc['ward']}

        # Nếu không đủ ward+district, thì match district
        for loc in locations:
            district = unidecode(loc['district'].lower())
            if re.search(rf'\b{re.escape(district)}\b', question):
                matched['district'] = loc['district']
                matched['city'] = loc['city']

        # Cuối cùng, chỉ match city nếu không có gì khác
        for loc in locations:
            city = unidecode(loc['city'].lower())
            if not matched['district'] and re.search(rf'\b{re.escape(city)}\b', question):
                matched['city'] = loc['city']

        return matched
    except Exception as e:
        print(f"[extract_location_from_question_v2] Error: {e}")
        return {'city': None, 'district': None, 'ward': None}



# ======= STEP 2: Sinh SQL từ model ========
def inference_sql_from_question(raw_question: str) -> str:
    """Sinh SQL từ câu hỏi tự nhiên bằng cách trích schema liên quan và gọi model."""
    try:
        question = normalize_question(raw_question)
        matched_location = extract_location_from_question_v2(question, locations)
        relevant_cols = extract_relevant_columns_from_question(question, full_schema)
        input_text = generate_input_text(question, relevant_cols)

        inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(
            inputs.input_ids,
            max_length=64,
            num_beams=1,
            #config.early_stopping = False,
            decoder_start_token_id=model.config.decoder_start_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

        raw_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
        sql = smart_fix_sql(raw_sql, question)
        sql = fix_location_in_sql(sql, matched_location)
        return sql
    except Exception as e:
        print(f"Error during inference: {e}")
        return ""


# ======= STEP 3: Fix SQL sau khi sinh ========
def smart_fix_sql(sql: str, question: str = "") -> str:
    """
    Apply heuristic fixes to SQL:
    - Fix missing ANDs
    - Collapse repeated words
    - Convert BETWEEN if "từ ... đến" is detected
    - Fix wrong comparison operators for phrases like 'dưới', 'cao hơn', etc.
    """
    try:
        sql = re.sub(r"(\d[\)']?)\s+([a-zA-Z_]+\s*=)", r"\1 AND \2", sql)
        sql = re.sub(r"(\d[\)']?)\s+([a-zA-Z_]+\s*[><])", r"\1 AND \2", sql)
        sql = re.sub(r"\b(\w+)\b(?:\s+\1\b)+", r"\1", sql)
        if "từ" in question and "đến" in question:
          m = re.search(r"price\s*>=\s*(\d+\.?\d*)\s*AND\s*price\s*<=\s*(\d+\.?\d*)", sql)
          if m:
              x, y = m.groups()
              sql = re.sub(
                  r"price\s*>=\s*\d+\.?\d*\s*AND\s*price\s*<=\s*\d+\.?\d*",
                  f"price BETWEEN {x} AND {y}",
                  sql
              )
        if any(k in question for k in ['dưới', 'ít hơn', 'rẻ hơn', 'thấp hơn']):
            sql = re.sub(r"(price|quantity)\s*>=\s*(\d+)", r"\1 < \2", sql)
        if any(k in question for k in ['trên', 'nhiều hơn', 'cao hơn', 'đắt hơn']):
            sql = re.sub(r"(price|quantity)\s*<=\s*(\d+)", r"\1 > \2", sql)
        return sql
    except Exception as e:
        print(f"Error in smart_fix_sql: {e}")
        return sql

def fix_location_in_sql(sql: str, matched_location: dict) -> str:
    try:
        sql = re.sub(r"(AND\s+)?(city|district|ward)\s*=\s*'[^']*'", "", sql, flags=re.IGNORECASE)
        sql = re.sub(r"\s+WHERE\s+AND", " WHERE ", sql, flags=re.IGNORECASE)
        sql = re.sub(r"\s+AND\s+AND", " AND ", sql)

        conditions = []
        if matched_location.get('ward'):
            conditions.append(f"ward = '{matched_location['ward']}'")
        if matched_location.get('district'):
            conditions.append(f"district = '{matched_location['district']}'")
        if matched_location.get('city'):
            conditions.append(f"city = '{matched_location['city']}'")

        if not conditions:
            return sql.strip()

        # Tìm vị trí để chèn WHERE trước ORDER BY / GROUP BY / LIMIT
        split_pattern = r"\b(order by|group by|limit|having)\b"
        parts = re.split(split_pattern, sql, flags=re.IGNORECASE)

        if "where" in sql.lower():
            parts[0] += " AND " + " AND ".join(conditions)
        else:
            parts[0] += " WHERE " + " AND ".join(conditions)

        return " ".join(parts).strip()
    except Exception as e:
        print(f"[fix_location_in_sql] Error: {e}")
        return sql

# ======= STEP 4: Thực hiện truy vấn SQLite ========

def run_query(sql):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.execute(sql)
        cols = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()
        conn.close()
        df = pd.DataFrame(rows, columns=cols)

        # Nếu quá nhiều dòng thì giới hạn preview
        return df.head(200) if len(df) > 200 else df
    except Exception as e:
        print(f"Error running SQL query: {e}")
        return pd.DataFrame(columns=["Lỗi"], data=[[str(e)]])


In [ ]:
# ====== GRADIO UI (Blocks layout) ======
with gr.Blocks(theme='soft') as demo:
    gr.Markdown("# 🏡 Text-to-SQL Real Estate Chatbot")
    gr.Markdown("Nhập câu hỏi tự nhiên, hệ thống sẽ trả kết quả từ CSDL bất động sản.")

    input_box = gr.Textbox(
        label="Câu hỏi của bạn",
        placeholder="Ví dụ: Tìm nhà dưới 3 tỷ ở Gò Vấp",
        lines=2
    )

    submit_btn = gr.Button("📤 Submit")

    result_table = gr.Dataframe(label="Kết quả truy vấn", interactive=False)

    # Hàm xử lý khi nhấn Submit (đặt bên trong block)
    def handle_query(user_input):
        print("** Bước 1: Chuẩn hoá câu hỏi...")
        normalized_q = normalize_question(user_input)
        print("Câu hỏi sau chuẩn hoá:", normalized_q)

        print("- Bước 2: Trích xuất địa danh...")
        matched_location = extract_location_from_question_v2(normalized_q, locations)
        print("Địa danh:", matched_location)

        # Chèn thêm print input tại đây nếu muốn
        relevant_cols = extract_relevant_columns_from_question(normalized_q, full_schema)
        input_text = generate_input_text(normalized_q, relevant_cols)
        print("→ Input cho mô hình:", input_text)

        print("- Bước 3: Sinh SQL từ mô hình...")
        raw_sql = inference_sql_from_question(user_input)
        print("SQL raw:", raw_sql)

        print("- Bước 4: Fix logic SQL...")
        sql_fixed = smart_fix_sql(raw_sql, normalized_q)
        print("SQL logic fixed:", sql_fixed)

        print("- Bước 5: Fix lại địa danh trong SQL...")
        final_sql = fix_location_in_sql(sql_fixed, matched_location)
        print("SQL final:", final_sql)

        print("- Bước 6: Thực hiện truy vấn và trả kết quả...")
        return run_query(final_sql)

    submit_btn.click(fn=handle_query, inputs=input_box, outputs=result_table)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://96722cc56bff158966.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


** Bước 1: Chuẩn hoá câu hỏi...
Câu hỏi sau chuẩn hoá: tìm nhà ở phú nhuận giá dưới 3 tỷ
- Bước 2: Trích xuất địa danh...
Địa danh: {'city': 'Hồ Chí Minh', 'district': 'Phú Nhuận', 'ward': None}
→ Input cho mô hình: Câu hỏi: tìm nhà ở phú nhuận giá dưới 3 tỷ | Schema: price[float]
- Bước 3: Sinh SQL từ mô hình...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


SQL raw: SELECT * FROM price_house WHERE price < 3000000000
- Bước 4: Fix logic SQL...
SQL logic fixed: SELECT * FROM price_house WHERE price < 3000000000
- Bước 5: Fix lại địa danh trong SQL...
SQL final: SELECT * FROM price_house WHERE price < 3000000000 AND district = 'Phú Nhuận' AND city = 'Hồ Chí Minh'
- Bước 6: Thực hiện truy vấn và trả kết quả...
** Bước 1: Chuẩn hoá câu hỏi...
Câu hỏi sau chuẩn hoá: tìm nhà ở phú nhuận giá dưới 3 tỷ, diện tích trên 30m2
- Bước 2: Trích xuất địa danh...
Địa danh: {'city': 'Hồ Chí Minh', 'district': 'Phú Nhuận', 'ward': None}
→ Input cho mô hình: Câu hỏi: tìm nhà ở phú nhuận giá dưới 3 tỷ, diện tích trên 30m2 | Schema: area[float], price[float]
- Bước 3: Sinh SQL từ mô hình...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


SQL raw: SELECT * FROM price_house WHERE price < 3000000000
- Bước 4: Fix logic SQL...
SQL logic fixed: SELECT * FROM price_house WHERE price < 3000000000
- Bước 5: Fix lại địa danh trong SQL...
SQL final: SELECT * FROM price_house WHERE price < 3000000000 AND district = 'Phú Nhuận' AND city = 'Hồ Chí Minh'
- Bước 6: Thực hiện truy vấn và trả kết quả...
** Bước 1: Chuẩn hoá câu hỏi...
Câu hỏi sau chuẩn hoá: tìm nhà ở hà nội giá dưới 2 tỷ diện tích trên 20m2
- Bước 2: Trích xuất địa danh...
Địa danh: {'city': 'Hà Nội', 'district': None, 'ward': None}
→ Input cho mô hình: Câu hỏi: tìm nhà ở hà nội giá dưới 2 tỷ diện tích trên 20m2 | Schema: area[float], price[float]
- Bước 3: Sinh SQL từ mô hình...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


SQL raw: SELECT * FROM price_house WHERE price < 2000000000 AND area > 20 AND city = 'Hà Nội'
- Bước 4: Fix logic SQL...
SQL logic fixed: SELECT * FROM price_house WHERE price < 2000000000 AND area > 20 AND city = 'Hà Nội'
- Bước 5: Fix lại địa danh trong SQL...
SQL final: SELECT * FROM price_house WHERE price < 2000000000 AND area > 20  AND city = 'Hà Nội'
- Bước 6: Thực hiện truy vấn và trả kết quả...
